In [9]:
# Final version for simple check data processing
import pandas as pd
import os

def process_excel_data(base_path, excel_filename):
    # Load the Excel file
    excel_path = os.path.join(base_path, excel_filename)
    sheets = pd.read_excel(excel_path, sheet_name=None)  # Load all sheets into a dictionary

    # Process each sheet
    for sheet_name, simple_check_df in sheets.items():
        # Identify the index where the second part starts by locating the header "P1W1"
        active_area_start_idx = simple_check_df[simple_check_df.eq("P1W1").any(axis=1)].index.min()

        # Extract the general information data and the active area data separately, making a copy to avoid SettingWithCopyWarning
        general_info_df = simple_check_df.iloc[:active_area_start_idx].copy()
        active_area_df = simple_check_df.iloc[active_area_start_idx + 1:].copy()  # skip the row with headers and make a copy
        active_area_df.columns = simple_check_df.iloc[active_area_start_idx]  # Set new header for active area data

        # Construct correct keys for the mapping
        general_info_df['Mapping_Key'] = general_info_df.apply(lambda x: f'P{x["Plate #"]}W{x["Well #"]}', axis=1)
        well_plate_genotype_mapping = general_info_df.set_index('Mapping_Key')[['Plate ID', 'Genotype']]

        # Create the new DataFrame structured as per the requirements
        new_csv_data = []
        for column in active_area_df.columns[2:]:  # skip 'Date' and 'DIV' columns
            for index, row in active_area_df.iterrows():
                if column in well_plate_genotype_mapping.index:
                    plate_id = well_plate_genotype_mapping.loc[column, 'Plate ID']
                    genotype = well_plate_genotype_mapping.loc[column, 'Genotype']
                    well_number = int(column.split('W')[1])  # Assuming well numbers are like 'W1', 'W2', etc.
                    plate_number = general_info_df[general_info_df['Mapping_Key'] == column]['Plate #'].values[0]  # Get the Plate #
                    # Standardize NeuronType for any genotype containing "WT"
                    neuron_type = "WT" if "WT" in genotype else genotype
                    new_csv_data.append({
                        'DIV': row['DIV'],
                        'Chip_ID': plate_id,
                        'Well': well_number,
                        'Plate_ID': plate_number,  # Add Plate_ID from the general info
                        'NeuronType': neuron_type,
                        'Active_area': row[column]
                    })

        # Convert list of dictionaries into a DataFrame
        new_csv_df = pd.DataFrame(new_csv_data)

        # Sort the DataFrame by 'DIV', 'Plate_ID', and 'Well'
        new_csv_df['Well'] = pd.to_numeric(new_csv_df['Well'])  # Ensure 'Well' is an integer
        new_csv_df.sort_values(by=['DIV', 'Plate_ID', 'Well'], ascending=[True, True, True], inplace=True)

        # Define the full path for saving the file
        full_path = os.path.join(base_path, sheet_name, 'Activity', 'Compiled_ActivityScan.csv')
        os.makedirs(os.path.dirname(full_path), exist_ok=True)  # Create the directory if it doesn't exist

        # Save the DataFrame to a new CSV file
        new_csv_df.to_csv(full_path, index=False)

    print("Data processing complete. Files have been saved in their respective directories.")

In [10]:
# Example usage:
base_path = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/QuickCheck'
excel_filename = 'simple_check.xlsx'
process_excel_data(base_path, excel_filename)

Data processing complete. Files have been saved in their respective directories.


In [9]:
# final version for generate data for homogenity check
import os
import pandas as pd

def adjust_and_copy_csvs(source_dir, destination_dir):
    # Ensure the destination directory exists
    os.makedirs(destination_dir, exist_ok=True)
    
    # Iterate over folders in the source directory
    for folder in os.listdir(source_dir):
        folder_path = os.path.join(source_dir, folder)
        if os.path.isdir(folder_path) and not folder.startswith('.'):
            # Define paths for activity and network CSVs
            activity_csv_path = os.path.join(folder_path, 'Compiled_ActivityScan.csv')
            network_csv_path = os.path.join(folder_path, 'Compiled_Networks.csv')
            output_folder = os.path.join(destination_dir, folder)
            os.makedirs(output_folder, exist_ok=True)  # Create corresponding folder in destination
            
            # Process Network CSV
            if os.path.exists(network_csv_path):
                df_network = pd.read_csv(network_csv_path)
                # Standardize 'NeuronType' values and strip spaces
                if 'NeuronType' in df_network.columns:
                    df_network['NeuronType'] = df_network['NeuronType'].str.strip().replace(regex={r'^.*WT.*$': 'WT'})
                # Rename 'IBI' column to 'mean_IBI' if it exists
                if 'IBI' in df_network.columns:
                    df_network.rename(columns={'IBI': 'mean_IBI'}, inplace=True)
                # Save the modified CSV to the destination path
                df_network.to_csv(os.path.join(output_folder, 'Compiled_Networks.csv'), index=False)
            
            # Process Activity CSV
            if os.path.exists(activity_csv_path):
                df_activity = pd.read_csv(activity_csv_path)
                # Standardize 'NeuronType' values and strip spaces
                if 'NeuronType' in df_activity.columns:
                    df_activity['NeuronType'] = df_activity['NeuronType'].str.strip().replace(regex={r'^.*WT.*$': 'WT'})
                # Rename 'Active_Electrodes' to 'Active_area' if necessary
                if 'Active_area' not in df_activity.columns and 'Active_Electrodes' in df_activity.columns:
                    df_activity.rename(columns={'Active_Electrodes': 'Active_area'}, inplace=True)
                # Save the modified CSV to the destination path
                df_activity.to_csv(os.path.join(output_folder, 'Compiled_ActivityScan.csv'), index=False)

def merge_activity_data_and_update_networks(homocheck_dir, quickcheck_dir):
    homo_folders = os.listdir(homocheck_dir)
    quick_folders = os.listdir(quickcheck_dir)

    for folder in homo_folders:
        homo_path = os.path.join(homocheck_dir, folder)
        activity_csv_path = os.path.join(homo_path, 'Compiled_ActivityScan.csv')
        network_csv_path = os.path.join(homo_path, 'Compiled_Networks.csv')
        
        # Update Compiled_Networks.csv if it exists
        if os.path.exists(network_csv_path):
            df_network = pd.read_csv(network_csv_path)
            if 'NeuronType' in df_network.columns:
                df_network['NeuronType'] = df_network['NeuronType'].str.strip().replace(regex={r'^.*WT.*$': 'WT'})
            df_network.to_csv(network_csv_path, index=False)
        
        if os.path.exists(activity_csv_path):
            df_homo = pd.read_csv(activity_csv_path)
            if 'Active_area' not in df_homo.columns and folder in quick_folders:
                quick_path = os.path.join(quickcheck_dir, folder, 'Activity', 'Compiled_ActivityScan.csv')
                if os.path.exists(quick_path):
                    df_quick = pd.read_csv(quick_path)
                    if 'Active_area' in df_quick.columns:
                        merged_df = pd.merge(df_homo, df_quick[['Well', 'DIV', 'Chip_ID', 'Active_area']],
                                             on=['Well', 'DIV', 'Chip_ID'], how='left')
                        merged_df.to_csv(activity_csv_path, index=False)

def process_datasets(source_dir, destination_dir, quickcheck_dir):
    # Adjust and copy datasets
    adjust_and_copy_csvs(source_dir, destination_dir)

    # Merge additional data into the datasets
    merge_activity_data_and_update_networks(destination_dir, quickcheck_dir)

In [10]:
# Example usage
source = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/CSVs'
destination = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/HomoCheck'
quickcheck = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/QuickCheck'

process_datasets(source, destination, quickcheck)

In [2]:
import pandas as pd

# Load the simple_check.csv to examine its contents
simple_check_path = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/simple_check.csv'
simple_check_df = pd.read_csv(simple_check_path)

# Identify the index where the second part starts by locating the header "P1W1"
active_area_start_idx = simple_check_df[simple_check_df.eq("P1W1").any(axis=1)].index.min()  # Corrected to specify axis as keyword argument

# Extract the general information data and the active area data separately
general_info_df = simple_check_df.iloc[:active_area_start_idx]
active_area_df = simple_check_df.iloc[active_area_start_idx + 1:]  # skip the row with headers
active_area_df.columns = simple_check_df.iloc[active_area_start_idx]  # Set new header for active area data

# Construct correct keys for the mapping
general_info_df['Mapping_Key'] = general_info_df.apply(lambda x: f'P{x["Plate #"]}W{x["Well #"]}', axis=1)
well_plate_genotype_mapping = general_info_df.set_index('Mapping_Key')[['Plate ID', 'Genotype']]

# Create the new DataFrame structured as per the requirements
new_csv_data = []
for column in active_area_df.columns[2:]:  # skip 'Date' and 'DIV' columns
    for index, row in active_area_df.iterrows():
        if column in well_plate_genotype_mapping.index:
            plate_id = well_plate_genotype_mapping.loc[column, 'Plate ID']
            genotype = well_plate_genotype_mapping.loc[column, 'Genotype']
            new_csv_data.append({
                'DIV': row['DIV'],
                'Chip_ID': plate_id,
                'Well': column,
                'NeuronType': genotype,
                'Active_area': row[column]
            })

# Convert list of dictionaries into a DataFrame
new_csv_df = pd.DataFrame(new_csv_data)

# Save the DataFrame to a new CSV file
new_csv_path = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/standardized_simple_check.csv'
new_csv_df.to_csv(new_csv_path, index=False)

/var/folders/hz/mtvvz9t52kxgqcx423vlrhx00000gn/T/ipykernel_70775/97710178.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_info_df['Mapping_Key'] = general_info_df.apply(lambda x: f'P{x["Plate #"]}W{x["Well #"]}', axis=1)


In [3]:
new_csv_df

,DIV,Chip_ID,Well,NeuronType,Active_area
0,7,M05506,P1W1,WT1,51.05
1,11,M05506,P1W1,WT1,80.65
2,14,M05506,P1W1,WT1,92.98
3,18,M05506,P1W1,WT1,95.61
4,21,M05506,P1W1,WT1,95.62
...,...,...,...,...,...
163,18,M07309,P5W3,WT2,93.73
164,21,M07309,P5W3,WT2,94.73
165,25,M07309,P5W3,WT2,92.56
166,27,M07309,P5W3,WT2,90.42


In [5]:
import pandas as pd

# Load the simple_check.csv to examine its contents
simple_check_path = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/simple_check.csv'
simple_check_df = pd.read_csv(simple_check_path)

# Identify the index where the second part starts by locating the header "P1W1"
active_area_start_idx = simple_check_df[simple_check_df.eq("P1W1").any(axis=1)].index.min()

# Extract the general information data and the active area data separately
general_info_df = simple_check_df.iloc[:active_area_start_idx]
active_area_df = simple_check_df.iloc[active_area_start_idx + 1:]  # skip the row with headers
active_area_df.columns = simple_check_df.iloc[active_area_start_idx]  # Set new header for active area data

# Construct correct keys for the mapping
general_info_df['Mapping_Key'] = general_info_df.apply(lambda x: f'P{x["Plate #"]}W{x["Well #"]}', axis=1)
well_plate_genotype_mapping = general_info_df.set_index('Mapping_Key')[['Plate ID', 'Genotype']]

# Create the new DataFrame structured as per the requirements
new_csv_data = []
for column in active_area_df.columns[2:]:  # skip 'Date' and 'DIV' columns
    for index, row in active_area_df.iterrows():
        if column in well_plate_genotype_mapping.index:
            plate_id = well_plate_genotype_mapping.loc[column, 'Plate ID']
            genotype = well_plate_genotype_mapping.loc[column, 'Genotype']
            well_number = column[-1]  # Extract the last character, which is the well number
            new_csv_data.append({
                'DIV': row['DIV'],
                'Chip_ID': plate_id,
                'Well': well_number,  # Use the extracted well number
                'NeuronType': genotype,
                'Active_area': row[column]
            })

# Convert list of dictionaries into a DataFrame
new_csv_df = pd.DataFrame(new_csv_data)

# Save the DataFrame to a new CSV file
new_csv_path = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/standardized_simple_check.csv'
new_csv_df.to_csv(new_csv_path, index=False)

/var/folders/hz/mtvvz9t52kxgqcx423vlrhx00000gn/T/ipykernel_70775/2877308360.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_info_df['Mapping_Key'] = general_info_df.apply(lambda x: f'P{x["Plate #"]}W{x["Well #"]}', axis=1)


In [6]:
import pandas as pd

# Load the simple_check.csv to examine its contents
simple_check_path = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/simple_check.csv'
simple_check_df = pd.read_csv(simple_check_path)

# Identify the index where the second part starts by locating the header "P1W1"
active_area_start_idx = simple_check_df[simple_check_df.eq("P1W1").any(axis=1)].index.min()

# Extract the general information data and the active area data separately
general_info_df = simple_check_df.iloc[:active_area_start_idx]
active_area_df = simple_check_df.iloc[active_area_start_idx + 1:]  # skip the row with headers
active_area_df.columns = simple_check_df.iloc[active_area_start_idx]  # Set new header for active area data

# Construct correct keys for the mapping
general_info_df['Mapping_Key'] = general_info_df.apply(lambda x: f'P{x["Plate #"]}W{x["Well #"]}', axis=1)
well_plate_genotype_mapping = general_info_df.set_index('Mapping_Key')[['Plate ID', 'Genotype']]

# Create the new DataFrame structured as per the requirements
new_csv_data = []
for column in active_area_df.columns[2:]:  # skip 'Date' and 'DIV' columns
    for index, row in active_area_df.iterrows():
        if column in well_plate_genotype_mapping.index:
            plate_id = well_plate_genotype_mapping.loc[column, 'Plate ID']
            genotype = well_plate_genotype_mapping.loc[column, 'Genotype']
            well_number = column[-1]  # Extract the last character, which is the well number
            # Standardize NeuronType for any genotype containing "WT"
            neuron_type = "WT" if "WT" in genotype else genotype
            new_csv_data.append({
                'DIV': row['DIV'],
                'Chip_ID': plate_id,
                'Well': well_number,  # Use the extracted well number
                'NeuronType': neuron_type,
                'Active_area': row[column]
            })

# Convert list of dictionaries into a DataFrame
new_csv_df = pd.DataFrame(new_csv_data)

# Save the DataFrame to a new CSV file
new_csv_path = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/Compiled_ActivityScan.csv'
new_csv_df.to_csv(new_csv_path, index=False)

/var/folders/hz/mtvvz9t52kxgqcx423vlrhx00000gn/T/ipykernel_70775/3689096833.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_info_df['Mapping_Key'] = general_info_df.apply(lambda x: f'P{x["Plate #"]}W{x["Well #"]}', axis=1)


In [27]:
# # spike only and full trace analysis
# import pandas as pd
# import os
# import re  # Import the regular expression library

# # Define the base path for saving the results
# base_path = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/SpikeOnly_FullTrace_check'

# # Load the Excel file
# excel_path = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/SpikeOnly_FullTrace_check/HET.xlsx'
# sheets = pd.read_excel(excel_path, sheet_name=None)  # Load all sheets into a dictionary

# # Process each sheet
# for sheet_name, simple_check_df in sheets.items():
#     # Identify the index where the second part starts by locating the header "P1W1"
#     active_area_start_idx = simple_check_df[simple_check_df.eq("P1W4").any(axis=1)].index.min()

#     # Extract the general information data and the active area data separately, making a copy to avoid SettingWithCopyWarning
#     general_info_df = simple_check_df.iloc[:active_area_start_idx].copy()
#     active_area_df = simple_check_df.iloc[active_area_start_idx + 1:].copy()  # skip the row with headers and make a copy
#     active_area_df.columns = simple_check_df.iloc[active_area_start_idx]  # Set new header for active area data

#     # Construct correct keys for the mapping
#     general_info_df['Mapping_Key'] = general_info_df.apply(lambda x: f'P{x["Plate #"]}W{x["Well #"]}', axis=1)
#     well_plate_genotype_mapping = general_info_df.set_index('Mapping_Key')[['Plate ID', 'Genotype']]

#     # Create the new DataFrame structured as per the requirements
#     new_csv_data = []
#     for column in active_area_df.columns[2:]:  # skip 'Date' and 'DIV' columns
#         for index, row in active_area_df.iterrows():
#             if column in well_plate_genotype_mapping.index:
#                 plate_id = well_plate_genotype_mapping.loc[column, 'Plate ID']
#                 genotype = well_plate_genotype_mapping.loc[column, 'Genotype']
#                 # Use regex to extract the well number (handles one or two digits)
#                 well_number = re.search('W(\d+)', column).group(1)
#                 # Standardize NeuronType for any genotype containing "WT"
#                 neuron_type = "WT" if "WT" in genotype else genotype
#                 new_csv_data.append({
#                     'DIV': row['DIV'],
#                     'Chip_ID': plate_id,
#                     'Well': well_number,
#                     'NeuronType': neuron_type,
#                     'Active_area': row[column]
#                 })

#     # Convert list of dictionaries into a DataFrame
#     new_csv_df = pd.DataFrame(new_csv_data)

#     # Define the full path for saving the file
#     full_path = os.path.join(base_path, sheet_name, 'Activity', 'Compiled_ActivityScan.csv')
#     os.makedirs(os.path.dirname(full_path), exist_ok=True)  # Create the directory if it doesn't exist

#     # Save the DataFrame to a new CSV file
#     new_csv_df.to_csv(full_path, index=False)

# print("Data processing complete. Files have been saved in their respective directories.")

Data processing complete. Files have been saved in their respective directories.


In [ ]:
import pandas as pd
import os

# Define the base path for saving the results
base_path = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/QuickCheck'

# Load the Excel file
excel_path = base_path + '/simple_check.xlsx'
sheets = pd.read_excel(excel_path, sheet_name=None)  # Load all sheets into a dictionary

# Process each sheet
for sheet_name, simple_check_df in sheets.items():
    # Identify the index where the second part starts by locating the header "P1W1"
    active_area_start_idx = simple_check_df[simple_check_df.eq("P1W1").any(axis=1)].index.min()

    # Extract the general information data and the active area data separately, making a copy to avoid SettingWithCopyWarning
    general_info_df = simple_check_df.iloc[:active_area_start_idx].copy()
    active_area_df = simple_check_df.iloc[active_area_start_idx + 1:].copy()  # skip the row with headers and make a copy
    active_area_df.columns = simple_check_df.iloc[active_area_start_idx]  # Set new header for active area data

    # Construct correct keys for the mapping
    general_info_df['Mapping_Key'] = general_info_df.apply(lambda x: f'P{x["Plate #"]}W{x["Well #"]}', axis=1)
    well_plate_genotype_mapping = general_info_df.set_index('Mapping_Key')[['Plate ID', 'Genotype']]

    # Create the new DataFrame structured as per the requirements
    new_csv_data = []
    for column in active_area_df.columns[2:]:  # skip 'Date' and 'DIV' columns
        for index, row in active_area_df.iterrows():
            if column in well_plate_genotype_mapping.index:
                plate_id = well_plate_genotype_mapping.loc[column, 'Plate ID']
                genotype = well_plate_genotype_mapping.loc[column, 'Genotype']
                well_number = column[-1]  # Extract the last character, which is the well number
                # Standardize NeuronType for any genotype containing "WT"
                neuron_type = "WT" if "WT" in genotype else genotype
                new_csv_data.append({
                    'DIV': row['DIV'],
                    'Chip_ID': plate_id,
                    'Well': well_number,
                    'NeuronType': neuron_type,
                    'Active_area': row[column]
                })

    # Convert list of dictionaries into a DataFrame
    new_csv_df = pd.DataFrame(new_csv_data)

    # Define the full path for saving the file
    full_path = os.path.join(base_path, sheet_name, 'Activity', 'Compiled_ActivityScan.csv')
    os.makedirs(os.path.dirname(full_path), exist_ok=True)  # Create the directory if it doesn't exist

    # Save the DataFrame to a new CSV file
    new_csv_df.to_csv(full_path, index=False)

print("Data processing complete. Files have been saved in their respective directories.")

In [ ]:
# add sorting function
import pandas as pd
import os

# Define the base path for saving the results
base_path = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/QuickCheck'

# Load the Excel file
excel_path = base_path + '/simple_check.xlsx'
sheets = pd.read_excel(excel_path, sheet_name=None)  # Load all sheets into a dictionary

# Process each sheet
for sheet_name, simple_check_df in sheets.items():
    # Identify the index where the second part starts by locating the header "P1W1"
    active_area_start_idx = simple_check_df[simple_check_df.eq("P1W1").any(axis=1)].index.min()

    # Extract the general information data and the active area data separately, making a copy to avoid SettingWithCopyWarning
    general_info_df = simple_check_df.iloc[:active_area_start_idx].copy()
    active_area_df = simple_check_df.iloc[active_area_start_idx + 1:].copy()  # skip the row with headers and make a copy
    active_area_df.columns = simple_check_df.iloc[active_area_start_idx]  # Set new header for active area data

    # Construct correct keys for the mapping
    general_info_df['Mapping_Key'] = general_info_df.apply(lambda x: f'P{x["Plate #"]}W{x["Well #"]}', axis=1)
    well_plate_genotype_mapping = general_info_df.set_index('Mapping_Key')[['Plate ID', 'Genotype']]

    # Create the new DataFrame structured as per the requirements
    new_csv_data = []
    for column in active_area_df.columns[2:]:  # skip 'Date' and 'DIV' columns
        for index, row in active_area_df.iterrows():
            if column in well_plate_genotype_mapping.index:
                plate_id = well_plate_genotype_mapping.loc[column, 'Plate ID']
                genotype = well_plate_genotype_mapping.loc[column, 'Genotype']
                well_number = int(column.split('W')[1])  # Assuming well numbers are like 'W1', 'W2', etc.
                # Standardize NeuronType for any genotype containing "WT"
                neuron_type = "WT" if "WT" in genotype else genotype
                new_csv_data.append({
                    'DIV': row['DIV'],
                    'Chip_ID': plate_id,
                    'Well': well_number,
                    'NeuronType': neuron_type,
                    'Active_area': row[column]
                })

    # Convert list of dictionaries into a DataFrame
    new_csv_df = pd.DataFrame(new_csv_data)
    
    # Sort the DataFrame by 'DIV' and 'Well' with Well treated as integer
    new_csv_df['Well'] = pd.to_numeric(new_csv_df['Well'])  # Ensure 'Well' is an integer
    new_csv_df.sort_values(by=['DIV', 'Well'], ascending=[True, True], inplace=True)

    # Define the full path for saving the file
    full_path = os.path.join(base_path, sheet_name, 'Activity', 'Compiled_ActivityScan.csv')
    os.makedirs(os.path.dirname(full_path), exist_ok=True)  # Create the directory if it doesn't exist

    # Save the DataFrame to a new CSV file
    new_csv_df.to_csv(full_path, index=False)

print("Data processing complete. Files have been saved in their respective directories.")

In [4]:
# add sorting function
import pandas as pd
import os

# Define the base path for saving the results
base_path = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/QuickCheck'

# Load the Excel file
excel_path = base_path + '/simple_check.xlsx'
sheets = pd.read_excel(excel_path, sheet_name=None)  # Load all sheets into a dictionary

# Process each sheet
for sheet_name, simple_check_df in sheets.items():
    # Identify the index where the second part starts by locating the header "P1W1"
    active_area_start_idx = simple_check_df[simple_check_df.eq("P1W1").any(axis=1)].index.min()

    # Extract the general information data and the active area data separately, making a copy to avoid SettingWithCopyWarning
    general_info_df = simple_check_df.iloc[:active_area_start_idx].copy()
    active_area_df = simple_check_df.iloc[active_area_start_idx + 1:].copy()  # skip the row with headers and make a copy
    active_area_df.columns = simple_check_df.iloc[active_area_start_idx]  # Set new header for active area data

    # Construct correct keys for the mapping
    general_info_df['Mapping_Key'] = general_info_df.apply(lambda x: f'P{x["Plate #"]}W{x["Well #"]}', axis=1)
    well_plate_genotype_mapping = general_info_df.set_index('Mapping_Key')[['Plate ID', 'Genotype']]

    # Create the new DataFrame structured as per the requirements
    new_csv_data = []
    for column in active_area_df.columns[2:]:  # skip 'Date' and 'DIV' columns
        for index, row in active_area_df.iterrows():
            if column in well_plate_genotype_mapping.index:
                plate_id = well_plate_genotype_mapping.loc[column, 'Plate ID']
                genotype = well_plate_genotype_mapping.loc[column, 'Genotype']
                well_number = int(column.split('W')[1])  # Assuming well numbers are like 'W1', 'W2', etc.
                plate_number = general_info_df[general_info_df['Mapping_Key'] == column]['Plate #'].values[0]  # Get the Plate #
                # Standardize NeuronType for any genotype containing "WT"
                neuron_type = "WT" if "WT" in genotype else genotype
                new_csv_data.append({
                    'DIV': row['DIV'],
                    'Chip_ID': plate_id,
                    'Well': well_number,
                    'Plate_ID': plate_number,  # Add Plate_ID from the general info
                    'NeuronType': neuron_type,
                    'Active_area': row[column]
                })

    # Convert list of dictionaries into a DataFrame
    new_csv_df = pd.DataFrame(new_csv_data)
    
    # Sort the DataFrame by 'DIV', 'Plate_ID', and 'Well'
    new_csv_df['Well'] = pd.to_numeric(new_csv_df['Well'])  # Ensure 'Well' is an integer
    new_csv_df.sort_values(by=['DIV', 'Plate_ID', 'Well'], ascending=[True, True, True], inplace=True)

    # Define the full path for saving the file
    full_path = os.path.join(base_path, sheet_name, 'Activity', 'Compiled_ActivityScan.csv')
    os.makedirs(os.path.dirname(full_path), exist_ok=True)  # Create the directory if it doesn't exist

    # Save the DataFrame to a new CSV file
    new_csv_df.to_csv(full_path, index=False)

print("Data processing complete. Files have been saved in their respective directories.")

Data processing complete. Files have been saved in their respective directories.


In [1]:
import os
import pandas as pd

def combine_and_process_csvs(base_path_1, base_path_2, output_path):
    # Ensure the output directory exists
    os.makedirs(output_path, exist_ok=True)
    
    # Collect all valid folder names from both directions to create corresponding folders in the output directory
    all_folders = set()
    for base_path in [base_path_1, base_path_2]:
        for folder in os.listdir(base_path):
            folder_path = os.path.join(base_path, folder)
            if os.path.isdir(folder_path) and not folder.startswith('.'):
                all_folders.add(folder)

    # Create folders in the output path
    for folder in all_folders:
        os.makedirs(os.path.join(output_path, folder), exist_ok=True)

    # Process the first direction
    for folder in os.listdir(base_path_1):
        folder_path = os.path.join(base_path_1, folder)
        if os.path.isdir(folder_path) and not folder.startswith('.'):
            # Check for the existence of CSV files
            activity_csv = os.path.join(folder_path, 'Compiled_ActivityScan.csv')
            network_csv = os.path.join(folder_path, 'Compiled_Networks.csv')
            
            # Process Activity CSV
            if os.path.exists(activity_csv):
                df_activity = pd.read_csv(activity_csv)
                # Standardize 'NeuronType' values
                df_activity['NeuronType'] = df_activity['NeuronType'].str.strip().replace(regex={r'^.*WT.*$': 'WT'})
                # Rename column if necessary
                if 'Active_Electrodes' in df_activity.columns:
                    df_activity.rename(columns={'Active_Electrodes': 'Active_area'}, inplace=True)
                
                # Save the modified CSV to the output path
                df_activity.to_csv(os.path.join(output_path, folder, 'Compiled_ActivityScan.csv'), index=False)

            # Process Network CSV
            if os.path.exists(network_csv):
                df_network = pd.read_csv(network_csv)
                # Rename 'IBI' column to 'mean_IBI' if it exists
                if 'IBI' in df_network.columns:
                    df_network.rename(columns={'IBI': 'mean_IBI'}, inplace=True)
                
                # Save the modified CSV to the output path
                df_network.to_csv(os.path.join(output_path, folder, 'Compiled_Networks.csv'), index=False)
    
    # Process the second direction
    for folder in os.listdir(base_path_2):
        folder_path = os.path.join(base_path_2, folder)
        if os.path.isdir(folder_path) and not folder.startswith('.'):
            activity_folder_path = os.path.join(folder_path, 'Activity')
            if os.path.exists(activity_folder_path):
                activity_csv = os.path.join(activity_folder_path, 'Compiled_ActivityScan.csv')
                if os.path.exists(activity_csv):
                    # Read and directly save the CSV to the output directory
                    df_activity = pd.read_csv(activity_csv)
                    df_activity.to_csv(os.path.join(output_path, folder, 'Compiled_ActivityScan.csv'), index=False)

# Example usage
base_path_1 = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/CSVs'
base_path_2 = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/QuickCheck'
output_path = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/HomoCheck'

combine_and_process_csvs(base_path_1, base_path_2, output_path)

In [2]:
import os
import pandas as pd

def combine_and_process_csvs(base_path_1, base_path_2, output_path):
    # Ensure the output directory exists
    os.makedirs(output_path, exist_ok=True)
    
    # Collect folder names from the CSVs directory (base_path_1) and create corresponding folders in the output directory
    valid_folders = [folder for folder in os.listdir(base_path_1) if os.path.isdir(os.path.join(base_path_1, folder)) and not folder.startswith('.')]
    for folder in valid_folders:
        os.makedirs(os.path.join(output_path, folder), exist_ok=True)

    # Process the first direction (CSVs directory)
    for folder in valid_folders:
        folder_path = os.path.join(base_path_1, folder)
        # Check for the existence of CSV files
        activity_csv = os.path.join(folder_path, 'Compiled_ActivityScan.csv')
        network_csv = os.path.join(folder_path, 'Compiled_Networks.csv')
        
        # Process Activity CSV
        if os.path.exists(activity_csv):
            df_activity = pd.read_csv(activity_csv)
            # Standardize 'NeuronType' values
            df_activity['NeuronType'] = df_activity['NeuronType'].str.strip().replace(regex={r'^.*WT.*$': 'WT'})
            # Rename column if necessary
            if 'Active_Electrodes' in df_activity.columns:
                df_activity.rename(columns={'Active_Electrodes': 'Active_area'}, inplace=True)
            
            # Save the modified CSV to the output path
            df_activity.to_csv(os.path.join(output_path, folder, 'Compiled_ActivityScan.csv'), index=False)

        # Process Network CSV
        if os.path.exists(network_csv):
            df_network = pd.read_csv(network_csv)
            # Rename 'IBI' column to 'mean_IBI' if it exists
            if 'IBI' in df_network.columns:
                df_network.rename(columns={'IBI': 'mean_IBI'}, inplace=True)
            
            # Save the modified CSV to the output path
            df_network.to_csv(os.path.join(output_path, folder, 'Compiled_Networks.csv'), index=False)
    
    # Process the second direction (QuickCheck directory)
    for folder in valid_folders:
        activity_folder_path = os.path.join(base_path_2, folder, 'Activity')
        if os.path.exists(activity_folder_path):
            activity_csv = os.path.join(activity_folder_path, 'Compiled_ActivityScan.csv')
            if os.path.exists(activity_csv):
                # Read and directly save the CSV to the output directory
                df_activity = pd.read_csv(activity_csv)
                df_activity.to_csv(os.path.join(output_path, folder, 'Compiled_ActivityScan.csv'), index=False)

# Example usage
base_path_1 = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/CSVs'
base_path_2 = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/QuickCheck'
output_path = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/HomoCheck'

combine_and_process_csvs(base_path_1, base_path_2, output_path)


In [5]:
import os
import pandas as pd

def adjust_and_copy_csvs(source_dir, destination_dir):
    # Ensure the destination directory exists
    os.makedirs(destination_dir, exist_ok=True)
    
    # Iterate over folders in the source directory
    for folder in os.listdir(source_dir):
        folder_path = os.path.join(source_dir, folder)
        if os.path.isdir(folder_path) and not folder.startswith('.'):
            # Define paths for activity and network CSVs
            activity_csv_path = os.path.join(folder_path, 'Compiled_ActivityScan.csv')
            network_csv_path = os.path.join(folder_path, 'Compiled_Networks.csv')
            output_folder = os.path.join(destination_dir, folder)
            os.makedirs(output_folder, exist_ok=True)  # Create corresponding folder in destination
            
            # Process Network CSV
            if os.path.exists(network_csv_path):
                df_network = pd.read_csv(network_csv_path)
                # Rename 'IBI' column to 'mean_IBI' if it exists
                if 'IBI' in df_network.columns:
                    df_network.rename(columns={'IBI': 'mean_IBI'}, inplace=True)
                # Save the modified CSV to the destination path
                df_network.to_csv(os.path.join(output_folder, 'Compiled_Networks.csv'), index=False)
            
            # Process Activity CSV
            if os.path.exists(activity_csv_path):
                df_activity = pd.read_csv(activity_csv_path)
                # Standardize 'NeuronType' values and strip spaces
                df_activity['NeuronType'] = df_activity['NeuronType'].str.strip().replace(regex={r'^.*WT.*$': 'WT'})
                # Rename 'Active_Electrodes' to 'Active_area' if necessary
                if 'Active_area' not in df_activity.columns and 'Active_Electrodes' in df_activity.columns:
                    df_activity.rename(columns={'Active_Electrodes': 'Active_area'}, inplace=True)
                # Save the modified CSV to the destination path
                df_activity.to_csv(os.path.join(output_folder, 'Compiled_ActivityScan.csv'), index=False)

# Example usage
source_dir = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/CSVs'
destination_dir = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/HomoCheck'

adjust_and_copy_csvs(source_dir, destination_dir)

In [8]:
import os
import pandas as pd

def adjust_and_copy_csvs(source_dir, destination_dir):
    # Ensure the destination directory exists
    os.makedirs(destination_dir, exist_ok=True)
    
    # Iterate over folders in the source directory
    for folder in os.listdir(source_dir):
        folder_path = os.path.join(source_dir, folder)
        if os.path.isdir(folder_path) and not folder.startswith('.'):
            # Define paths for activity and network CSVs
            activity_csv_path = os.path.join(folder_path, 'Compiled_ActivityScan.csv')
            network_csv_path = os.path.join(folder_path, 'Compiled_Networks.csv')
            output_folder = os.path.join(destination_dir, folder)
            os.makedirs(output_folder, exist_ok=True)  # Create corresponding folder in destination
            
            # Process Network CSV
            if os.path.exists(network_csv_path):
                df_network = pd.read_csv(network_csv_path)
                # Standardize 'NeuronType' values and strip spaces
                if 'NeuronType' in df_network.columns:
                    df_network['NeuronType'] = df_network['NeuronType'].str.strip().replace(regex={r'^.*WT.*$': 'WT'})
                # Rename 'IBI' column to 'mean_IBI' if it exists
                if 'IBI' in df_network.columns:
                    df_network.rename(columns={'IBI': 'mean_IBI'}, inplace=True)
                # Save the modified CSV to the destination path
                df_network.to_csv(os.path.join(output_folder, 'Compiled_Networks.csv'), index=False)
            
            # Process Activity CSV
            if os.path.exists(activity_csv_path):
                df_activity = pd.read_csv(activity_csv_path)
                # Standardize 'NeuronType' values and strip spaces
                if 'NeuronType' in df_activity.columns:
                    df_activity['NeuronType'] = df_activity['NeuronType'].str.strip().replace(regex={r'^.*WT.*$': 'WT'})
                # Rename 'Active_Electrodes' to 'Active_area' if necessary
                if 'Active_area' not in df_activity.columns and 'Active_Electrodes' in df_activity.columns:
                    df_activity.rename(columns={'Active_Electrodes': 'Active_area'}, inplace=True)
                # Save the modified CSV to the destination path
                df_activity.to_csv(os.path.join(output_folder, 'Compiled_ActivityScan.csv'), index=False)

# Example usage
source_dir = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/CSVs'
destination_dir = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/HomoCheck'

adjust_and_copy_csvs(source_dir, destination_dir)

In [6]:
import os
import pandas as pd

def merge_activity_data(homocheck_dir, quickcheck_dir):
    homo_folders = os.listdir(homocheck_dir)
    quick_folders = os.listdir(quickcheck_dir)

    for folder in homo_folders:
        homo_path = os.path.join(homocheck_dir, folder)
        activity_csv_path = os.path.join(homo_path, 'Compiled_ActivityScan.csv')
        
        if os.path.exists(activity_csv_path):
            df_homo = pd.read_csv(activity_csv_path)
            
            if 'Active_area' not in df_homo.columns:
                # Check if there is a corresponding folder in QuickCheck
                if folder in quick_folders:
                    quick_path = os.path.join(quickcheck_dir, folder, 'Activity', 'Compiled_ActivityScan.csv')
                    
                    if os.path.exists(quick_path):
                        df_quick = pd.read_csv(quick_path)
                        if 'Active_area' in df_quick.columns:
                            # Perform the merge based on 'Well', 'DIV', and 'Chip_ID'
                            merged_df = pd.merge(df_homo, df_quick[['Well', 'DIV', 'Chip_ID', 'Active_area']],
                                                 on=['Well', 'DIV', 'Chip_ID'], how='left')
                            
                            if merged_df['Active_area'].isnull().all():
                                print(f"No rows to match in {folder} based on 'Well', 'DIV', and 'Chip_ID'.")
                            else:
                                # Save back to HomoCheck folder
                                merged_df.to_csv(activity_csv_path, index=False)
                                print(f"Updated 'Active_area' for {folder}.")
                        else:
                            print(f"No 'Active_area' column found in QuickCheck for folder {folder}.")
                    else:
                        print(f"No matching CSV found in QuickCheck for folder {folder}.")
                else:
                    print(f"No matching folder found in QuickCheck for folder {folder}.")
        else:
            print(f"No Activity CSV found in HomoCheck for folder {folder}.")

# Example usage
homocheck_dir = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/HomoCheck'
quickcheck_dir = '/Users/liufanling/Library/CloudStorage/OneDrive-Personal/1 UC DAVIS/2024 Summer/CSRA/QualityCheck/QuickCheck'

merge_activity_data(homocheck_dir, quickcheck_dir)

No Activity CSV found in HomoCheck for folder .DS_Store.
Updated 'Active_area' for ADNP_T2_10262023.
Updated 'Active_area' for ADNP_T3_11072023.
